In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import collections
import json
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
#def dictionary

In [3]:
def tokenize(line,token='word'):
    if token == 'word':
        return [line.split(' ')]
    elif token == 'char':
        return [list(line)]
    else:
        print('ERROR: unknown token type '+token)

In [4]:
def count_tokens(tokanized_sentences):
    # Flatten a list of token lists into a list of tokens
    tokens = [tk for line in tokanized_sentences for tk in line]
    return collections.Counter(tokens)

In [5]:
filename = 'MLDS_hw2_1_data/training_label.json'
with open(filename, 'r') as f:
    datastore = json.load(f)

In [6]:
vid_feat_set = {}
vid_sentence_set = {}

sizeof_train = 0
for data in datastore:
    video_id = data["id"]
    vid_feat_set[video_id]=None
    vid_sentence_set[video_id]=None
    sizeof_train = sizeof_train+1

In [7]:
batch_size = 50

batches = 1450/batch_size

vid_batch = {}

i = 0
j = 0
for data in datastore:
    vid_feat_list = []
    video_id = data["id"]
    features = np.load("MLDS_hw2_1_data/training_data/feat/{}.npy".format(video_id))
        
    vid_framefeats = []
    for array in features:
        vid_framefeats.append(array)
                
    if j not in vid_batch:
        vid_batch[j] = []

    vid_batch[j].append(vid_framefeats)

    #vid_feat_set[video_id] = vid_framefeats
    #vid_feat_list = np.append(vid_feat_list, vid_framefeats,axis=2)
    i = i+1
     
    if i%batch_size == 0:
        j = j+1

# vid_batches is the video features divided into batches of 50

In [8]:
i = 0
j = 0

sentence_set = {}

for data in datastore:
    
    video_id = data["id"]

    
    #print("reading sentences in: %s" % video_id)
    sentences = data["caption"]
    sentences = [word.lower() for word in sentences] #Normalize the case
    table = str.maketrans('', '', string.punctuation) #Normalize the punctuation
    sentences = [word.translate(table) for word in sentences]
    #print(sentences[0])
    
    sentence_set[i] = sentences[0]
    i = i +1
#Extracting only a single sentence per video

In [9]:
#sentence_set
print("The number of videos in the training set are %d and each video has 80 frames with 4096 features/units each" % len(vid_feat_set))

The number of videos in the training set are 1450 and each video has 80 frames with 4096 features/units each


In [10]:
# print(vid_sentence_set)


In [11]:
# #vid_feat_set['WqQonRVs7WA_0_10.avi']

# count = 0
# for x in vid_sentence_set: 
#     if isinstance(vid_sentence_set[x], list): 
#         count += len(vid_sentence_set[x]) 
# print(count) 

In [12]:
# Mapping string tokens to numertical indices.
def listVocab(sentence_set):
    
    PAD_token = 0
    BOS_token = 1
    EOS_token = 2
    UNK_token = 3
    
    all_tokens = []
    token_index = {"<PAD>": 0,"<BOS>":1,"<EOS>":2,"<UNK>":3}
    index_token = {PAD_token: "<PAD>", BOS_token: "<BOS>", EOS_token: "<EOS>", UNK_token: "<UNK>"}
    
    #for set_i in vid_sentence_set:
    #    sentence_set = vid_sentence_set[set_i]
    #    for line in sentence_set: 
    
    for n in sentence_set:
        line = sentence_set[n]
        tokenized_captions = tokenize(line) #Seperate the words
        all_tokens += tokenized_captions
    
    counter = count_tokens(all_tokens) #Count the word repeatitions in each set
    
    counter_dict = counter.items()
    counter_sort = sorted(counter_dict, key=lambda x:x[1],reverse=True) #sort by frequency of occurance 
    #print(counter_sort)

    i = len(index_token)
    values = [0,1,2,3]
    for token, freq in counter_sort:
        index_token[i] = token
        token_index[token] = i
        values += [i]
        i+=1
    
    return [values,token_index, index_token, len(index_token)]

In [13]:
values,token_index, index_token, nums = listVocab(sentence_set)
print("There are %d unique words in the captions dataset" % nums)

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


There are 1988 unique words in the captions dataset


In [14]:
#counter_dict

In [15]:
#np.shape(onehot_encoded)

In [16]:
def flattenList(nestedList,output): 
    for i in nestedList: 
        if type(i) == list: 
            flattenList(i,output) 
        else: 
            output.append(i) 
            
    return output

In [17]:

def num_encode(test_sentence,index_token,tokenized_sentence=[],num_encoded_sentence=[],onehot_encoded_sentence=[]):
    
    tokenized_sentence.clear()
    num_encoded_sentence.clear()
    onehot_encoded_sentence.clear()
    
    tokenized_sentence = ["<BOS>"] + tokenize(test_sentence) + ["<EOS>"]
    #print(tokenized_sentence)
    output=[]
    tokenized_sentence = flattenList(tokenized_sentence,output)

    while len(tokenized_sentence) < MAX_WORDS:
        tokenized_sentence.append("<PAD>")
    #print(len(tokenized_sentence))
   
    for token in tokenized_sentence:
        for i in range(0,len(index_token)):
            if token == index_token[i]: 
                num_encoded_sentence.append(i) 
                onehot_encoded_sentence.append(onehot_encoded[i])
        
    return tokenized_sentence, num_encoded_sentence, onehot_encoded_sentence


In [18]:
MAX_WORDS = 80

intencode_batch = {}
onehot_batch = {}

ii = 0
jj = 0

# for data in datastore:
#     video_id = data["id"]
#     num_encoded_dict[video_id]=None
#     onehot_encoded_dict[video_id]=None
  
    
for n in sentence_set:
    sentence = sentence_set[n]

    _,encoded_sentence,onehot_encoded_sentence = num_encode(sentence,index_token)
    
    encoded_sentence = list(encoded_sentence)
    onehot_encoded_sentence = list(onehot_encoded_sentence)
    

    #print(type(encoded_sentence)) 
    #print("nxt")
    #num_encoded_per_set.append(encoded_sentence)
    #print(num_encoded_per_set)


    #onehot_encoded_per_set.append(onehot_encoded_sentence)

    #print(num_encoded_per_set)
    
    #num_encoded_dict[vid] = num_encoded_per_set
    #onehot_encoded_dict[vid] = onehot_encoded_per_set
    
    if jj not in onehot_batch:
        onehot_batch[jj] = []
        #intencode_batch[j] = []

    #print(np.shape(onehot_encoded_sentence))    
    onehot_batch[jj].append(onehot_encoded_sentence)
    #intencode_batch[j].append(encoded_sentence)

    ii = ii+1
     
    if ii%batch_size == 0:
        jj = jj+1


In [26]:
from tensorflow.contrib import rnn

n_hidden = 1000
n_words = nums
sizeof_frame = 4096
no_of_frames = 80
sizeof_sentence= 80
learning_rate = 0.01

weights1 = tf.Variable(tf.random_normal([n_hidden,n_words]))
bias1 = tf.Variable(tf.zeros([n_words]))

#weights2 = tf.Variable(tf.random_normal([n_hidden,n_words]))
#bias2 = tf.Variable(tf.zeros([n_words]))

x_video = tf.placeholder(tf.float32, [None, no_of_frames, sizeof_frame])
y_label = tf.placeholder(tf.int32,[None, sizeof_sentence,n_words])

def RNN(x, weights1, biases1):
    
    x = tf.unstack(x,no_of_frames,1)
    
    lstm_encoder = tf.keras.layers.LSTM(n_hidden, return_state=True) #reuse=tf.AUTO_REUSE)
    output_encoder,state_h,state_c = lstm_encoder(x) #,dtype=tf.float32)
    encoder_states = [state_h,state_c]
    
    decoder
    
    return tf.matmul(output1[-1],weights1) + bias1
    

In [28]:
logits = RNN(x_video,weights1,bias1)
prediction = tf.nn.softmax(logits)


loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_label))


optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

2


In [29]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [31]:
# batch_y = onehot_batch[0]
# np.shape(batch_y)

In [33]:

loss_list = []
acc_list = []

with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    for i in range(29):
        batch_x = np.array(vid_batch[i])
        batch_y = np.array(onehot_batch[i])
        
        sess.run(train_op, feed_dict={x_video: batch_x, y_label: batch_y})
        
        loss, acc = sess.run([loss_op, accuracy], feed_dict={x_video: batch_x, y_label: batch_y})
        
        loss_list =+ loss
        acc_list =+ acc

InvalidArgumentError: logits and labels must be broadcastable: logits_size=[100,1988] labels_size=[4000,1988]
	 [[node softmax_cross_entropy_with_logits_sg_3 (defined at /home/adhitir/anaconda3/envs/tf_class/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'softmax_cross_entropy_with_logits_sg_3':
  File "/home/adhitir/anaconda3/envs/tf_class/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/adhitir/anaconda3/envs/tf_class/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/adhitir/anaconda3/envs/tf_class/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/adhitir/anaconda3/envs/tf_class/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/adhitir/anaconda3/envs/tf_class/lib/python3.6/site-packages/ipykernel/kernelapp.

In [ ]:
batch_x = np.array(vid_batch[0])
batch_y = np.array(onehot_batch[0])

with tf.Session() as sess:
    sess.run(init)

    sess.run(train_op, feed_dict={x_video: batch_x, y_label: batch_y})

In [ ]:


image_emb = tf.nn.xw_plus_b(x_video, weights1, bias1) 
#image_emb = tf.reshape(image_emb, [batch_size, no_of_frames, n_hidden])

#lstm2 = tf.keras.layers.LSTMCell(n_hidden)

padding = tf.zeros([batch_size, n_hidden])


#Only read the frames


        

            
                
logit_words = tf.nn.xw_plus_b(output2, weights2, bias2)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logit_words,onehot_encoded)

loss = tf.reduce_sum(cross_entropy)

In [ ]:
with tf.Session() as sess:
    with sess.as_default():
        print(tf.nn.embedding_lookup(onehot_encoded,[1]).eval())

In [ ]:
inputs